#### Run the following code to load the unprocessed real-life data in batches and store all of it into an HDF5 file that's easier to load

In [ ]:
#  Import the real target data from the RecogniChess local folder (../chess-dataset/labeled_originals) 

# Define the shape of the resized images (aspect ratio is 6:4, so we need to maintain it in the formula below)
resized_shape = (2048, int(2048 * 4000 / 6000))

# Batch loading of images
for i, filename in enumerate(os.listdir(path)):
    if filename.endswith('.jpg') or filename.endswith('.JPG'):
        img_path = os.path.join(path, filename)
        img = cv2.imread(img_path)
        target_dataset.append(img)
        
        # If batch size is reached, process the batch and clear the list
        if len(target_dataset) == batch_size or i == len(os.listdir(path))-1:
            # Process batch
            print(f"Processing batch {i//batch_size+1}")
            # Resize the images
            resized_batch = []
            for image in tqdm(target_dataset, desc="Resizing images"):
                resized_image = cv2.resize(image, resized_shape)
                resized_batch.append(resized_image)
                
            # Append resized images to the full dataset
            resized_dataset.extend(resized_batch)
            
            # Clear the list
            target_dataset = []

In [ ]:
# Convert the dataset to a numpy array and save to HDF5 file
with h5py.File('resized_dataset.hdf5', 'w') as f:
    dset = f.create_dataset("target_images", data=np.array(resized_dataset))

#### RE-START KERNEL ####

In [ ]:
# Load the target dataset from the HDF5 file in the RecogniChess local folder
with h5py.File('resized_dataset.hdf5', 'r') as f:
    resized_dataset = np.array(f['target_images'][:])

In [ ]:
# Display one image to verify it worked
display(Image(data=cv2.imencode('.jpg', resized_dataset[323])[1].tobytes(), format='jpeg'))